In [ ]:
import unittest
from unittest.mock import MagicMock, patch
import docx
from your_module import (
    add_title, add_corp_partecipants, add_conf_partecipants,
    clean_data_from_infer_errors, detect_umbrella_concept,
    add_fin_table, infer_data, add_bullets, add_heading
)

class TestDocumentFunctions(unittest.TestCase):

    def setUp(self):
        self.doc = MagicMock()
        self.doc.add_paragraph.return_value = MagicMock()
        self.doc.add_table.return_value = MagicMock()
        self.mock_paragraph = self.doc.add_paragraph.return_value
        self.mock_run = self.mock_paragraph.add_run.return_value
        self.mock_run.font = MagicMock()

    def test_add_title(self):
        global title
        title = "Q2 Report"
        add_title(self.doc)
        self.mock_run.add_text.assert_not_called()  # add_run handles text
        self.assertTrue(self.mock_run.bold)
        self.assertEqual(self.mock_run.font.name, 'Arial')

    def test_detect_umbrella_concept(self):
        self.assertEqual(detect_umbrella_concept("ARR growth"), "sales")
        self.assertEqual(detect_umbrella_concept("operating income margin"), "profit")
        self.assertEqual(detect_umbrella_concept("unknown"), "")

    def test_clean_data_from_infer_errors(self):
        mock_entity = MagicMock()
        mock_entity.ents = [MagicMock(label_="REPORTED VALUE", text="100")]
        data = [{'entity': mock_entity}]
        result = clean_data_from_infer_errors(data)
        self.assertEqual(len(result), 1)

    @patch("your_module.detect_umbrella_concept", return_value="profit")
    def test_add_fin_table(self, mock_detect):
        item = {'speaker': "Analyst", 'entity': MagicMock(text="Earnings call")}
        item['entity'].ents = [MagicMock(label_="REPORTED VALUE", text="500")]
        add_fin_table(self.doc, [item])
        self.doc.add_table.assert_called()

    @patch("your_module.nlp")
    def test_infer_data(self, mock_nlp):
        mock_nlp.return_value.ents = [MagicMock()]
        global parags
        parags = [{'speaker': "CEO", 'paragraph': ["Revenue was up 10%."]}]
        list_pres, list_qanda = infer_data()
        self.assertEqual(len(list_pres), 1)
        self.assertEqual(len(list_qanda), 0)

    def test_add_heading(self):
        add_heading("TITLE")
        self.doc.add_paragraph.assert_called()

if __name__ == "__main__":
    unittest.main()
